### import library

In [10]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl

from tqdm.notebook import tqdm
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import seed_everything, LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger

/home/hoahoang/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### custom dataset

In [11]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [12]:
# augmentation use transformer
dataset_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((300,300)),
        transforms.RandomRotation(50,expand=True),  
        transforms.Resize((300,300)),
        transforms.RandomCrop((120,120)),
        transforms.RandomVerticalFlip(0.4),
        transforms.RandomHorizontalFlip(0.4),                     
        transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0), 
        transforms.ToTensor(),
        AddGaussianNoise(0.1, 0.08),
        transforms.RandomErasing(),
    ]),
    'valid': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((300, 300)),
        transforms.ToTensor()])
}

In [13]:
class PetDataset(Dataset):
    def __init__(self, file_txt, root_dir, transform=None):
        with open(file_txt, 'r') as f:
            self.img_infos = f.readlines()
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_infos)

    def __getitem__(self, idx):
        img_info = self.img_infos[idx]
        img_info = img_info.split(' ')
        img_name = img_info[0]
        
        img_label = torch.zeros(37)
        img_label[int(img_info[1]) - 1] = 1.
        img_path = os.path.join(self.root_dir, img_name + '.jpg')
        img = cv2.imread(img_path)
        if self.transform:
            img = self.transform(img)

        return img, img_label

In [14]:
cd ..

/home/hoahoang


In [15]:
cd /home/hoahoang/training/data/oxford-iiit-pet

/home/hoahoang/training/data/oxford-iiit-pet


In [16]:
dataset_path = {
    'train': PetDataset(file_txt='./annotations/trainval.txt', root_dir='./images', transform=dataset_transforms['train']),
    'valid': PetDataset(file_txt='./annotations/test.txt', root_dir='./images', transform=dataset_transforms['valid']),
}

### build a model

In [17]:
class OurModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, batch_size=16):
        super().__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(2048, 37)
        )
        self.batch_size = batch_size
    
    def forward(self, x):
        x = self.base_model(x)
        return x

    def configure_optimizers(self):
        optimizer = optim.Adam([   
                {'params': list(model.parameters())[:-1], 'lr': 1e-4},
                {'params': list(model.parameters())[-1], 'lr': 5e-2}
                ])
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        return [optimizer], [scheduler]

    def configure_callbacks(self):
        early_stopping = EarlyStopping(monitor='train_loss', mode='min', patience=5, verbose=True, min_delta=0.001)
        checkpoint_callback = ModelCheckpoint(monitor='val_acc', mode='max', dirpath='./', save_top_k=1)
        return [early_stopping, checkpoint_callback]
    
    def train_dataloader(self):
        return DataLoader(dataset_path['train'], batch_size=self.batch_size, shuffle=True, num_workers=16)


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss(reduction='mean')(y_hat, y)
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        return {'loss': loss}

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.CrossEntropyLoss(reduction='mean')(y_hat, y)
        pred = torch.softmax(y_hat, dim=1)
        y = y.int()
        acc = accuracy(pred, y)
        return loss, acc
    
    def val_dataloader(self):
        return DataLoader(dataset_path['valid'], batch_size=self.batch_size, shuffle=False, num_workers=16)
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics, prog_bar=True, on_epoch=True)
        return metrics
    
    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     loss = F.cross_entropy(self(x), y)
    #     return {'val_loss': loss, 'log': {'val_loss': loss}}
    
    def validation_epoch_end(self, outputs):
        val_loss_mean = sum([o['val_loss'] for o in outputs]) / len(outputs)
        # show val_acc in progress bar but only log val_loss
        results = {'progress_bar': {'val_loss': val_loss_mean.item()}, 'log': {'val_loss': val_loss_mean.item()},
                   'val_loss': val_loss_mean.item()}
        return results

    def load_state_dict(self, state_dict):
        self.base_model.load_state_dict(state_dict)

    def state_dict(self):
        return self.base_model.state_dict()

### trainer

In [18]:
logger = TensorBoardLogger("runs", name="resnet50")

In [53]:
# few functions of trainer: auto_lr_find, accumulate_grad_batches, limit_training_batches, num_sanity_val_steps

In [55]:
if __name__ == '__main__':
    early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=5, verbose=True)
    checkpoint_callback = ModelCheckpoint(monitor='val_acc', mode='max', dirpath='./', save_top_k=1)
    model = OurModel()
    trainer = Trainer(max_epochs=10, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,logger=logger,
                      progress_bar_refresh_rate=10, callbacks=[early_stop_callback], checkpoint_callback=[checkpoint_callback],)
    trainer.tune(model)
    trainer.fit(model)
    save(model.state_dict(), 'Saved model')

/home/hoahoang/env/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=[<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ff6857e8710>])` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=[<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ff6857e8710>])`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, ModelCheckpoint
Missing logger folder: runs/resnet50

  | Name       | Type   | Params
--------------------------------------
0 | base_model | ResNet 

Epoch 0:  30%|███████████████████████████                                                              | 140/460 [01:18<03:00,  1.77it/s, loss=3.12, v_num=0, train_loss_step=3.280]

Corrupt JPEG data: premature end of data segment


Epoch 0:  37%|████████████████████████████████▉                                                        | 170/460 [01:34<02:42,  1.79it/s, loss=2.99, v_num=0, train_loss_step=2.710]

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


...

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


Epoch 9:  50%|█████████████████                 | 230/460 [51:35<51:35, 13.46s/it, loss=2.04, v_num=0, train_loss_step=2.060, val_acc=0.974, val_loss=2.460, train_loss_epoch=2.020]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████| 460/460 [54:57<00:00,  7.17s/it, loss=2.04, v_num=0, train_loss_step=2.060, val_acc=0.971, val_loss=3.130, train_loss_epoch=1.990]

Metric train_loss improved by 0.031 >= min_delta = 0.001. New best score: 1.986


Epoch 9: 100%|██████████████████████████████████| 460/460 [54:57<00:00,  7.17s/it, loss=2.04, v_num=0, train_loss_step=2.060, val_acc=0.971, val_loss=3.130, train_loss_epoch=1.990]


NameError: name 'save' is not defined

In [57]:
trainer.validate()

/home/hoahoang/env/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1448: UserWarning: `.validate(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `validate(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, ModelCheckpoint
Restoring states from the checkpoint path at /home/hoahoang/training/data/oxford-iiit-pet/epoch=6-step=1610-v1.ckpt
Loaded model weights from checkpoint at /home/hoahoang/training/data/oxford-iiit-pet/epoch=6-step=1610-v1.ckpt


Validation DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [03:27<00:00,  1.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.9744840264320374
        val_loss             2.155937433242798
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_acc': 0.9744840264320374, 'val_loss': 2.155937433242798}]

### visualization

In [56]:
! tensorboard --logdir=runs --load_fast=false --bind_all

TensorFlow installation not found - running with reduced feature set.
TensorBoard 2.8.0 at http://labserver:6009/ (Press CTRL+C to quit)
^C
